<a href="https://colab.research.google.com/github/ZainAli24/OpenAI_Agents_SDK_class_02/blob/main/Class_16_OpenAI_Agents_SDK_Input_Guardrails_and_Output_Guardrails_How_to_implement_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Guardrails in OpenAI-Agents SDK:**


---

### **Guardrails Kya Hain?**
Guardrails ka matlab hai ke hum apne agentic system ke **input** aur **output** ko kuch rules ya nizam ke mutabiq monitor aur control karte hain. Yeh is liye zaroori hai taake humara system sirf wahi kaam kare jo uska maqsad hai, aur na ke resources waste hon na hi galat jawab user tak pahunche. OpenAI Agents SDK mein guardrails do tarah ke hote hain: **Input Guardrails** aur **Output Guardrails**.

---

### **Input Guardrails: Pehla Guard Jo Input Ko Rokta Hai**
Hum ne maan liya ke ek teaching agent banaya hai jo sirf **Agentic AI** se related sawalon ka jawab deta hai. Is agent mein hum ne ek powerful aur expensive language model (LLM) istemal kiya hai. Ab agar user aisa input de jo is maqsad se hat kar ho, maslan **math ka sawal** pooche, toh agar yeh input seedha LLM tak chala gaya aur us ne is par process karke jawab diya, toh hamare **tokens waste** honge aur humein nuqsaan bhi ho sakta hai, kyunki yeh agent math ke liye nahi bana.

Is problem se bachne ke liye hum input ke raste mein ek **guard** lagate hain, yani ek **Input Guardrail Agent**. Yeh agent user ke input ko LLM tak jane se pehle check karta hai. Yeh dekhta hai ke kya yeh input Agentic AI se related hai ya nahi. Agar input relevant hai, toh woh LLM ke paas jata hai process hone ke liye. Agar nahi, toh yeh guard usko rok deta hai. Toh is tarah input guardrail ek filter ka kaam karta hai, jo irrelevant sawalon ko system mein dakhil hone se pehle hi rok deta hai. Is se resources bachte hain aur agent apne asal maqsad par focus karta hai.

---

### **Output Guardrails: Dusra Guard Jo Output Ko Theek Karta Hai**
Ab agar input relevant hai aur LLM ne usko process karke jawab diya, toh bhi kabhi kabhi woh jawab aisa hota hai jo hum user ko nahi dikhana chahte. Maslan, jawab **irrelevant** ho sakta hai, koi **sensitive ya political baat** shamil ho sakti hai, ya phir tone woh na ho jo hum chahte hain. Toh is ke liye hum output ke raste mein bhi ek **guard** lagate hain, yani **Output Guardrail Agent**.

Yeh output guardrail agent LLM ke jawab ko monitor karta hai aur yeh ensure karta hai ke jo response user tak jata hai woh:
- **Topic par ho** (off-topic na jaye),
- **Suitable ho** (koi galat ya nuqsandeh baat na ho),
- Aur **humare standards ke mutabiq ho** (jaise tone ya style).

Agar output mein koi masla hota hai, toh yeh guard usko theek kar sakta hai ya rok sakta hai, taake user tak sirf wahi jawab jaye jo sahi aur maqool ho.

---

### **Mukhtasar: Guardrails Ka Faida**
Guardrails ka concept ek tarah ka **quality control** hai jo agentic system ko behtar banata hai:
- **Input Guardrails** irrelevant sawalon ko rok kar resources aur tokens bachaate hain, taake LLM sirf zaruri kaam par laga rahe.
- **Output Guardrails** jawab ko polish karke yeh ensure karte hain ke user ko wahi mile jo hum chahte hain, aur koi galat ya ghair-zaroori cheez na jaye.

Yeh dono mil kar system ko efficient banate hain, user experience ko behtar karte hain, aur agent ko uske maqsad ke mutabiq chalne mein madad dete hain. Toh guardrails se hum apne agent ko na sirf tez aur kam kharcheela banate hain, balki uske jawab ko bhi reliable aur maqool rakhte hain.

---


# **2. Input & Output Guardrails:**

Aapka sawal OpenAI Agents SDK ke guardrails ke behind-the-scenes working ko samajhne ka hai, jahan aap input aur output guardrails ke functions aur classes (jaise `guardrail function`, `GuardrailFunctionOutput`, `InputGuardrailResult`, `.tripwire_triggered`, aur `InputGuardrailTripwireTriggered`) ke kaam karne ke tareeke ko aur clear karna chahte hain. Aapka basic concept sahi hai, lekin isko aur improve aur wazeh karne ke liye main aapko step-by-step samjhaunga aur code ke saath classes ki working bhi clear karunga.

---

### **Aapka Concept: Sahi Hai Ya Nahi?**
Aap ne jo samjha hai:
- Guardrails ek function hai jo agent ke andar hota hai.
- **Input Guardrail**: Starting agent (jo input leta hai) ke andar hota hai, input ko check karta hai, agar valid hai toh loop chalta hai, agar nahi toh tripwire ke zariye exception raise karke loop rok deta hai.
- **Output Guardrail**: Final output dene wale agent pe lagta hai, aur woh bhi same process follow karta hai.

Yeh concept **bilkul sahi hai**! Aap ne guardrails ka high-level flow theek samajh liya hai. Lekin aapko functions aur classes ke behind-the-scenes kaam ka detail chahiye, toh ab main isko code ke saath aur gehrayi mein samjhaata hoon.

---

### **Guardrails Ki Behind-the-Scenes Working**
Guardrails OpenAI Agents SDK mein input aur output ko monitor aur control karne ke liye mechanisms hain. Yeh ensure karte hain ke agent sirf valid inputs pe kaam kare aur appropriate outputs hi user tak pahunchen. Ab isko step-by-step dekhte hain.

#### **1. Input Guardrails**
- **Kahan Hota Hai?**: Input guardrail us agent ke andar hota hai jo user se input leta hai (starting agent). Yeh agent ke process ka pehla step hota hai.
- **Kaise Kaam Karta Hai?**:
  1. User input deta hai (e.g., "What’s the weather like?").
  2. Yeh input pehle **input guardrail function** mein jata hai, jo agent ke andar hota hai.
  3. Guardrail function input ko check karta hai (maslan, kya yeh agent ke scope mein hai? Koi harmful content toh nahi?).
  4. Agar input valid hai, toh ek result return hota hai jo agent ko aage process karne deta hai.
  5. Agar input invalid hai (e.g., irrelevant ya harmful), toh tripwire trigger hota hai, ek exception raise hoti hai, aur process ruk jata hai.

- **Classes aur Functions Involved**:
  - **`Guardrail Function`**: Yeh ek function hota hai (jaise `check_input`) jo input ko check karta hai.
  - **`InputGuardrailResult`**: Yeh class guardrail ke result ko store karti hai, jaise `valid=True` ya `valid=False`, aur `tripwire_triggered` property.
  - **`.tripwire_triggered`**: Yeh ek property hai jo batati hai ke guardrail check fail hua hai (True) ya nahi (False).
  - **`InputGuardrailTripwireTriggered`**: Yeh ek exception class hai jo tab raise hoti hai jab input invalid ho.

**Code Example**:
```python
class InputGuardrailResult:
    def __init__(self, valid, tripwire_triggered=False):
        self.valid = valid
        self.tripwire_triggered = tripwire_triggered

class InputGuardrailTripwireTriggered(Exception):
    pass

class InputGuardrail:
    def check_input(self, input_data):
        # Input check logic (e.g., length, content, relevance)
        if "valid" in input_data:  # Dummy check, real logic SDK mein hoga
            return InputGuardrailResult(valid=True)
        else:
            return InputGuardrailResult(valid=False, tripwire_triggered=True)

class Agent:
    def __init__(self):
        self.input_guardrail = InputGuardrail()

    def process_input(self, input_data):
        result = self.input_guardrail.check_input(input_data)
        if result.tripwire_triggered:
            raise InputGuardrailTripwireTriggered("Invalid input: " + input_data)
        else:
            print("Processing input:", input_data)

# Test
agent = Agent()
agent.process_input("valid input")  # Output: Processing input: valid input
agent.process_input("invalid")      # Raises: InputGuardrailTripwireTriggered
```

- **Explanation**:
  - `check_input` input ko check karta hai aur `InputGuardrailResult` object return karta hai.
  - Agar `tripwire_triggered=True`, toh `InputGuardrailTripwireTriggered` exception raise hoti hai, jo loop/process ko rok deta hai.

#### **2. Output Guardrails**
- **Kahan Hota Hai?**: Output guardrail us agent ke andar hota hai jo final output deta hai. Yeh process ka aakhri step hota hai.
- **Kaise Kaam Karta Hai?**:
  1. Agent input process karke output banata hai.
  2. Yeh output **output guardrail function** mein jata hai.
  3. Guardrail function output ko check karta hai (e.g., kya yeh safe aur relevant hai?).
  4. Agar output valid hai, toh user tak chala jata hai.
  5. Agar invalid hai, toh tripwire trigger hota hai, aur output rok diya jata hai ya modify kiya jata hai.

- **Classes aur Functions Involved**:
  - **`Guardrail Function`**: Yeh function output ko check karta hai (jaise `check_output`).
  - **`GuardrailFunctionOutput`**: Yeh shayad ek base class hai jo guardrail result ko represent karti hai (input aur output dono ke liye common ho sakti hai).
  - **`OutputGuardrailResult`**: Output check ka result store karti hai (SDK mein specific naam alag ho sakta hai).
  - **`.tripwire_triggered`**: Output check fail hone pe True hota hai.
  - **`OutputGuardrailTripwireTriggered`**: Exception class output invalid hone pe raise hoti hai.

**Code Example**:
```python
class OutputGuardrailResult:
    def __init__(self, valid, tripwire_triggered=False):
        self.valid = valid
        self.tripwire_triggered = tripwire_triggered

class OutputGuardrailTripwireTriggered(Exception):
    pass

class OutputGuardrail:
    def check_output(self, output_data):
        # Output check logic (e.g., appropriateness, safety)
        if "appropriate" in output_data:  # Dummy check
            return OutputGuardrailResult(valid=True)
        else:
            return OutputGuardrailResult(valid=False, tripwire_triggered=True)

class Agent:
    def __init__(self):
        self.output_guardrail = OutputGuardrail()

    def generate_output(self, processed_data):
        output = "Output: " + processed_data
        result = self.output_guardrail.check_output(output)
        if result.tripwire_triggered:
            raise OutputGuardrailTripwireTriggered("Inappropriate output: " + output)
        else:
            return output

# Test
agent = Agent()
print(agent.generate_output("appropriate data"))  # Output: Output: appropriate data
agent.generate_output("inappropriate")            # Raises: OutputGuardrailTripwireTriggered
```

- **Explanation**:
  - `check_output` output ko check karta hai aur `OutputGuardrailResult` return karta hai.
  - Agar `tripwire_triggered=True`, toh `OutputGuardrailTripwireTriggered` exception raise hoti hai.

---

### **Classes Ka Role**
Aapke mention kiye classes ke behind-the-scenes kaam:
- **`GuardrailFunctionOutput`**: Yeh ek base ya generic class ho sakti hai jo guardrail function ke result ko encapsulate karti hai. Input aur output dono ke liye common ho sakti hai.
- **`InputGuardrailResult`**: Specifically input guardrail ke result ko store karti hai, jisme `valid` aur `tripwire_triggered` jaise attributes hote hain.
- **`.tripwire_triggered`**: Yeh property result object mein hoti hai aur batati hai ke guardrail check fail hua ya nahi.
- **`InputGuardrailTripwireTriggered`**: Yeh exception class hai jo invalid input/output ke case mein raise hoti hai.

---

### **Complete Flow**
1. **Input**:
   - User → Input → `InputGuardrail.check_input()` → `InputGuardrailResult`:
     - `valid=True` → Agent processing.
     - `tripwire_triggered=True` → `InputGuardrailTripwireTriggered` → Stop.
2. **Output**:
   - Agent → Output → `OutputGuardrail.check_output()` → `OutputGuardrailResult`:
     - `valid=True` → User ko output.
     - `tripwire_triggered=True` → `OutputGuardrailTripwireTriggered` → Stop.

---

### **Aapka Concept Improved**
Aapka concept sahi tha, lekin ab code aur classes ki detail ke saath yeh aur clear ho gaya hai:
- Guardrails agent ke andar functions hote hain jo input/output ko check karte hain.
- Yeh classes ke objects banate hain (jaise `InputGuardrailResult`) jo result store karte hain.
- Tripwire mechanism invalid cases ko handle karta hai exceptions ke zariye.



# **3. Guardrails Practically apply through Code**

# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.2/444.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.2 MB/s eta 0:00:00


## Make your Notebook capable of running asynchronous functions.

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Implementation of Input Guardrail:
Checks that the input going into your LLM is "safe"

In [3]:
from pydantic import BaseModel
from agents import (
    Agent,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    RunContextWrapper,
    TResponseInputItem,

    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    input_guardrail,
    output_guardrail,
)
from google.colab import userdata


In [4]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

In [5]:
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,
)


config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

## **Structured Output:**

In [6]:
# from agents import enable_verbose_stdout_logging

# enable_verbose_stdout_logging()

In [7]:
class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str
    answer: str   # user question answer

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
    model=model
)

In [8]:
result = Runner.run_sync(
    guardrail_agent,
    "What is the capital of Pakistan",
)

result.final_output

MathHomeworkOutput(is_math_homework=False, reasoning='Islamabad is the capital city of Pakistan.', answer='Islamabad')

In [9]:
print("is_math_homework:", result.final_output.is_math_homework)
print("reasoning:", result.final_output.reasoning)
print("answer:", result.final_output.answer)


is_math_homework: False
reasoning: Islamabad is the capital city of Pakistan.
answer: Islamabad


In [10]:
result = Runner.run_sync(
    guardrail_agent,
    "what is output of 22 + 55 - 11",
)

result.final_output

MathHomeworkOutput(is_math_homework=True, reasoning='22 + 55 = 77, 77 - 11 = 66', answer='66')

----------------------------

## **1. Upper code working Flow:**

Aapka sawal bilkul wazeh hai, aur aapne code ka concept kaafi had tak theek samjha hai. Main aapke samajhne ke tareeke ko analyze karunga, aapki confusion ko clear karunga, aur guardrail agent ke backend working ko code ke saath aur detail mein samjhaunga, taake aapko yeh samajh aaye ke `output_type` kaise kaam karta hai aur LLM ko kaise pata chala ke `is_math_homework=False` aur reasoning kyun di.

---

### **Aapka Concept: Sahi Hai Ya Nahi?**
Aap ne jo samjha hai, usko summarize karte hain:
1. **Pydantic BaseModel**: Aapne kaha ke `MathHomeworkOutput` class `BaseModel` se inherit karta hai, jo output ke type ko validate karta hai. Yeh bilkul sahi hai.
2. **Agent Creation**: Aapne kaha ke `Agent` banaya gaya hai jiska kaam hai user input ko check karna ke kya woh math homework se related hai, aur `output_type=MathHomeworkOutput` se agent ko bataya gaya ke output is format mein hona chahiye. Yeh bhi sahi hai.
3. **Runner.run_sync**: Aapne kaha ke `Runner.run_sync` se agent run hota hai, aur output mein `is_math_homework=False` aur reasoning aata hai. Yeh bhi theek hai.
4. **Confusion**: Aapko confusion hai ke:
   - Agent ko kaise pata chala ke `is_math_homework=False` hona chahiye aur reasoning kyun di?
   - LLM ke paas toh sirf system prompt, user prompt, tool schema, aur tool output jata hai, lekin `output_type` kaise kaam karta hai jab yeh explicitly LLM ke paas nahi jata?

Main ab yeh step-by-step samjhaunga, aur aapki confusion ko clear karunga.

---

### **Code Analysis aur Backend Working**
Aapka diya hua code:

```python
from pydantic import BaseModel

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)  

result = Runner.run_sync(
    guardrail_agent,
    "What is the capital of Islamabad",
)

result.final_output
# Output: MathHomeworkOutput(is_math_homework=False, reasoning='The user is asking a question about geography, not math.')
```

Ab isko break down karte hain.

---

#### **1. `MathHomeworkOutput` Class (Pydantic BaseModel)**
- **Kya Hai?**: Yeh ek Pydantic `BaseModel` class hai jo output ka structure define karti hai. Ismein do fields hain:
  - `is_math_homework`: Boolean (True/False) jo batata hai ke input math homework se related hai ya nahi.
  - `reasoning`: String jo batati hai ke yeh decision kyun liya gaya.
- **Kaam**: Yeh class output ko validate karta hai aur ensure karta hai ke agent ka output is specific format mein ho (bool aur string ke saath).
- **Behind the Scenes**: Pydantic ka `BaseModel` type checking aur validation ke liye istemal hota hai. Jab agent output deta hai, Pydantic check karta hai ke output is structure ke mutabiq hai ya nahi. Agar nahi hai, toh error raise hota hai.

---

#### **2. `Agent` Creation**
- **Kya Hai?**:
  ```python
  guardrail_agent = Agent(
      name="Guardrail check",
      instructions="Check if the user is asking you to do their math homework.",
      output_type=MathHomeworkOutput,
  )
  ```
  Yeh ek guardrail agent hai jo input ko analyze karta hai aur check karta hai ke kya user math homework ke sawal pooch raha hai.
- **Parameters**:
  - `name`: Agent ka naam, jo identification ke liye hai.
  - `instructions`: System prompt jo LLM ko batata hai ke input ko kaise process karna hai (yani, check karo ke math homework hai ya nahi).
  - `output_type`: Yeh batata hai ke output ka format `MathHomeworkOutput` class ke mutabiq hona chahiye.
- **Kaam**: Agent input ko LLM ke through process karta hai aur output ko `MathHomeworkOutput` format mein return karta hai.

---

#### **3. `Runner.run_sync`**
- **Kya Hai?**: `Runner.run_sync` ek method hai jo agent ko synchronously run karta hai. Yeh agent ko input deta hai ("What is the capital of Islamabad") aur output collect karta hai.
- **Kaam**: Yeh agent ke execution ko handle karta hai, LLM se baat karta hai, aur final output return karta hai.
- **Output**: `result.final_output` ek `MathHomeworkOutput` object hai jo `is_math_homework=False` aur `reasoning="The user is asking a question about geography, not math."` deta hai.

---

#### **4. Aapki Confusion: LLM Ko Kaise Pata Chala?**
Aapka sawal hai ke:
- LLM ke paas toh sirf system prompt, user prompt, tool schema, aur tool output jata hai. Toh `output_type` kaise kaam karta hai?
- Agent ko kaise pata chala ke `is_math_homework=False` aur reasoning kyun di?

Let’s clear this step-by-step.

##### **LLM Ko Kya Milta Hai?**
Jab `Runner.run_sync` agent ko run karta hai, yeh process hota hai:
1. **System Prompt**: Agent ka `instructions` parameter system prompt ban jata hai:
   ```
   Check if the user is asking you to do their math homework.
   ```
   Yeh LLM ko batata hai ke input ko analyze karke yeh decide karna hai ke kya yeh math homework hai.

2. **User Prompt**: User ka input, yani:
   ```
   What is the capital of Islamabad
   ```

3. **Output Format (Schema)**: `output_type=MathHomeworkOutput` ka schema (Pydantic model ka structure) LLM ko bheja jata hai. Yeh schema batata hai ke output ek JSON object hona chahiye jismein:
   - `is_math_homework`: Boolean
   - `reasoning`: String
   Yeh schema explicitly LLM ke paas nahi dikhta code mein, lekin OpenAI Agents SDK ke backend mein yeh JSON schema ke roop mein LLM ko diya jata hai (tool schema ke tarah).

4. **LLM Ka Kaam**: LLM system prompt aur user prompt ko padhta hai, aur schema ke mutabiq output deta hai. Yani, LLM:
   - Input ("What is the capital of Islamabad") ko dekhta hai.
   - System prompt ke mutabiq analyze karta hai ke yeh math homework nahi hai, kyunki yeh geography ka sawal hai.
   - Schema ke mutabiq ek structured output deta hai:
     ```json
     {
         "is_math_homework": false,
         "reasoning": "The user is asking a question about geography, not math."
     }
     ```

##### **Output Type Kaise Kaam Karta Hai?**
- **Backend Working**:
  - `output_type=MathHomeworkOutput` Pydantic model ka schema banata hai, jo OpenAI Agents SDK ke backend mein JSON schema mein convert hota hai.
  - Yeh JSON schema LLM ko as a **tool schema** diya jata hai. OpenAI ke modern models (jaise GPT-4) structured output ke liye JSON schema ko samajh sakte hain.
  - LLM is schema ke mutabiq output generate karta hai, aur SDK is output ko Pydantic model (`MathHomeworkOutput`) mein parse karta hai.
  - Pydantic phir validate karta hai ke output sahi format mein hai ya nahi. Agar format galat hai, toh error raise hota hai.

- **Aapki Confusion Ka Jawab**:
  - Aapne kaha ke LLM ke paas sirf system prompt, user prompt, tool schema, aur tool output jata hai. Yeh sahi hai, lekin `output_type` ka schema **tool schema** ke roop mein LLM ko milta hai. Yani, `MathHomeworkOutput` ka structure (bool aur string fields) LLM ko bataya jata hai ke output aise hona chahiye.
  - SDK yeh schema internally handle karta hai, isliye aapko code mein directly nahi dikhta.

##### **LLM Ko Kaise Pata Chala `is_math_homework=False`?**
- **Analysis**:
  - System prompt ("Check if the user is asking you to do their math homework") LLM ko batata hai ke input ko analyze karna hai.
  - User input ("What is the capital of Islamabad") geography ka sawal hai, jo math se related nahi.
  - LLM apni reasoning capability se yeh samajhta hai ke yeh math homework nahi hai, aur isliye `is_math_homework=False` set karta hai.
  - Reasoning field mein LLM apni wajah deta hai, jo schema ke mutabiq string format mein hoti hai: "The user is asking a question about geography, not math."

---

### **Complete Backend Flow**
1. **Agent Initialization**:
   - `Agent` banate waqt `output_type=MathHomeworkOutput` se SDK ko pata chalta hai ke output ka structure kya hona chahiye.
   - `instructions` system prompt ban jata hai.

2. **Runner.run_sync**:
   - User input aur system prompt LLM ko bheje jate hain.
   - `MathHomeworkOutput` ka JSON schema (internally generated) LLM ko as a tool schema diya jata hai.
   - LLM input ko analyze karta hai aur schema ke mutabiq structured output deta hai:
     ```json
     {
         "is_math_homework": false,
         "reasoning": "The user is asking a question about geography, not math."
     }
     ```

3. **Output Validation**:
   - SDK is JSON output ko `MathHomeworkOutput` object mein parse karta hai.
   - Pydantic validate karta hai ke output sahi format mein hai (bool aur string).
   - Validated output `result.final_output` ke roop mein return hota hai.

---

### **Improved Explanation**
Aapka concept bilkul sahi tha, bas aapko `output_type` ke backend working ki wajah se confusion thi. Yeh clear karte hain:
- `output_type` directly LLM ko nahi jata, balki SDK isko JSON schema mein convert karta hai, jo LLM ke tool schema ke roop mein jata hai.
- LLM system prompt aur user input ke basis par reasoning karta hai, aur schema ke mutabiq structured output deta hai.
- SDK phir is output ko Pydantic model mein parse karta hai aur validate karta hai.

---

### **Aapki Confusion Ka Final Jawab**
- **LLM ko kaise pata chala?**: LLM ne system prompt ("Check if the user is asking you to do their math homework") aur input ("What is the capital of Islamabad") ko analyze kiya. Uski training aur reasoning capability ne yeh decide kiya ke yeh math nahi, geography hai. Isliye `is_math_homework=False` aur reasoning di.
- **Output_type kaise kaam karta hai?**: Yeh SDK ke zariye JSON schema ke roop mein LLM ko bheja jata hai, jo LLM ko batata hai ke output structured (JSON format) mein dena hai. SDK phir isko Pydantic model mein convert karta hai.

---

### **Example with Another Input**
Agar input hota: "Solve 2x + 3 = 7":
```python
result = Runner.run_sync(
    guardrail_agent,
    "Solve 2x + 3 = 7",
)
print(result.final_output)
# Output: MathHomeworkOutput(is_math_homework=True, reasoning='The user is asking to solve a mathematical equation.')
```
- **Kyun?**: LLM ne input ko math equation ke roop mein pehchana, isliye `is_math_homework=True` aur reasoning di.

---

Agar aapko kisi aur specific part (jaise JSON schema kaise banta hai ya Pydantic ka validation) pe aur detail chahiye, toh batayein!

## **2. Upper code backend logic flow:**

Aapka sawal OpenAI Agents SDK ke backend working ke bare mein hai, khas tor par yeh ke `output_type=MathHomeworkOutput` kaise JSON schema banata hai, yeh schema LLM ko kaise jata hai, aur SDK output ko Pydantic model mein kaise parse karta hai. Aapne jo backend working ka concept quote kiya hai, usko main aur wazeh karunga, parsing ka matlab samjhaunga, aur code ke saath output ke sath is process ko detail mein explain karunga.

---

### **Key Concepts to Understand**
1. **JSON Schema**: Yeh ek structure hai jo batata hai ke data (output) kaisa hona chahiye (maslan, koi field boolean hona chahiye ya string).
2. **Parsing**: Parsing ka matlab hai raw data (jaise JSON string) ko kisi specific format ya object mein convert karna. Yani, LLM ka output (JSON format mein) ko Pydantic model (`MathHomeworkOutput`) ke object mein badalna.
3. **Pydantic Validation**: Pydantic ek library hai jo ensure karti hai ke data (output) us model ke structure ke mutabiq hai. Agar data galat format mein hai, toh error raise hota hai.
4. **SDK ka Role**: OpenAI Agents SDK yeh sab automate karta hai—JSON schema banata hai, LLM ko bhejta hai, aur output ko parse karke Pydantic model mein convert karta hai.

Ab isko step-by-step code aur output ke saath samjhte hain.

---

### **Backend Working Explained**
Aapke code ke hisaab se:

```python
from pydantic import BaseModel

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)

result = Runner.run_sync(
    guardrail_agent,
    "What is the capital of Islamabad",
)

print(result.final_output)
# Output: MathHomeworkOutput(is_math_homework=False, reasoning='The user is asking a question about geography, not math.')
```

Ab yeh process kaise hota hai, dekhte hain.

---

#### **Step 1: `output_type=MathHomeworkOutput` se JSON Schema Banta Hai**
- **Kya Hota Hai?**:
  - Jab aap `output_type=MathHomeworkOutput` dete hain, OpenAI Agents SDK Pydantic model (`MathHomeworkOutput`) ka structure padhta hai.
  - Yeh structure JSON schema mein convert hota hai. JSON schema ek standard format hai jo data ke structure ko define karta hai.
  - `MathHomeworkOutput` ka JSON schema kuch aisa hoga:
    ```json
    {
      "type": "object",
      "properties": {
        "is_math_homework": {"type": "boolean"},
        "reasoning": {"type": "string"}
      },
      "required": ["is_math_homework", "reasoning"]
    }
    ```
  - Is schema mein bataya gaya hai ke output ek JSON object hona chahiye jismein:
    - `is_math_homework`: Boolean (True/False)
    - `reasoning`: String
    - Dono fields zaroori hain (`required`).

- **Output Connection**: Yeh schema batata hai ke LLM ka output is format mein hona chahiye. Yani, jab LLM output generate karega, usko is schema ke mutabiq JSON object banana hoga.

---

#### **Step 2: JSON Schema LLM ko Jata Hai**
- **Kya Hota Hai?**:
  - OpenAI Agents SDK yeh JSON schema LLM ko as a **tool schema** bhejta hai. Modern OpenAI models (jaise GPT-4) structured output ke liye JSON schema ko samajh sakte hain.
  - LLM ko teen cheezein milti hain:
    1. **System Prompt**: Agent ka `instructions` parameter, yani:
       ```
       Check if the user is asking you to do their math homework.
       ```
    2. **User Prompt**: User ka input, yani:
       ```
       What is the capital of Islamabad
       ```
    3. **Tool Schema**: Upar wala JSON schema, jo batata hai ke output ka format kya hona chahiye.
- **LLM Ka Kaam**:
  - LLM system prompt aur user prompt ko padhta hai.
  - Input ("What is the capital of Islamabad") ko analyze karta hai aur dekhta hai ke yeh math homework nahi, geography ka sawal hai.
  - Schema ke mutabiq ek JSON output generate karta hai:
    ```json
    {
      "is_math_homework": false,
      "reasoning": "The user is asking a question about geography, not math."
    }
    ```

- **Output Connection**: Yeh JSON object LLM ka raw output hai, jo schema ke mutabiq structured hai. Yani, LLM ne schema padha aur uske hisaab se `is_math_homework` aur `reasoning` fields bhare.

---

#### **Step 3: Parsing - SDK Output ko Pydantic Model Mein Convert Karta Hai**
- **Parsing Ka Matlab**:
  - Parsing ka matlab hai raw data (jaise JSON string ya object) ko kisi specific format mein convert karna. Yahan, LLM ka JSON output:
    ```json
    {
      "is_math_homework": false,
      "reasoning": "The user is asking a question about geography, not math."
    }
    ```
    ko `MathHomeworkOutput` Pydantic object mein convert kiya jata hai.
  - Yani, JSON ke fields (`is_math_homework`, `reasoning`) ko Python object ke attributes mein map kiya jata hai.

- **SDK Kaise Parsing Karta Hai?**:
  - OpenAI Agents SDK LLM se JSON output leta hai.
  - SDK is JSON ko `MathHomeworkOutput` class ke instance mein convert karta hai using Pydantic’s parsing functionality.
  - Code ke hisaab se, yeh process aisa dikhta hai:
    ```python
    from pydantic import BaseModel

    class MathHomeworkOutput(BaseModel):
        is_math_homework: bool
        reasoning: str

    # Maan lo yeh LLM ka raw JSON output hai
    json_output = {
        "is_math_homework": False,
        "reasoning": "The user is asking a question about geography, not math."
    }

    # SDK yeh karta hai (internally)
    pydantic_object = MathHomeworkOutput(**json_output)
    print(pydantic_object)
    # Output: MathHomeworkOutput(is_math_homework=False, reasoning='The user is asking a question about geography, not math.')
    ```

- **Behind the Scenes**:
  - Pydantic ka `BaseModel` class JSON data ko padhta hai aur usko Python object mein convert karta hai.
  - `**json_output` syntax Python mein dictionary ko unpack karta hai, taake `is_math_homework` aur `reasoning` fields directly `MathHomeworkOutput` ke attributes ban jayein.

---

#### **Step 4: Pydantic Validation**
- **Kya Hota Hai?**:
  - Pydantic `MathHomeworkOutput` object ke data ko validate karta hai:
    - Check karta hai ke `is_math_homework` boolean hai ya nahi.
    - Check karta hai ke `reasoning` string hai ya nahi.
    - Agar koi field missing hai ya galat type ka hai, toh Pydantic error raise karta hai.
  - Example agar galat output aaye:
    ```python
    json_output = {
        "is_math_homework": "not a boolean",  # Galat type
        "reasoning": "Some reason"
    }
    pydantic_object = MathHomeworkOutput(**json_output)  # Error: "is_math_homework" must be a boolean
    ```

- **Output Connection**:
  - Jab validation successful hoti hai, tab `result.final_output` ek valid `MathHomeworkOutput` object hota hai:
    ```python
    MathHomeworkOutput(is_math_homework=False, reasoning='The user is asking a question about geography, not math.')
    ```
  - Agar validation fail hoti hai, toh SDK error raise karta hai, aur output user tak nahi jata.

---

### **Complete Flow with Output**
1. **Input**: User deta hai: "What is the capital of Islamabad".
2. **Agent Setup**:
   - `MathHomeworkOutput` ka JSON schema banaya jata hai:
     ```json
     {
       "type": "object",
       "properties": {
         "is_math_homework": {"type": "boolean"},
         "reasoning": {"type": "string"}
       },
       "required": ["is_math_homework", "reasoning"]
     }
     ```
3. **LLM Processing**:
   - System prompt: "Check if the user is asking you to do their math homework."
   - User prompt: "What is the capital of Islamabad"
   - LLM schema ke mutabiq JSON output deta hai:
     ```json
     {
       "is_math_homework": false,
       "reasoning": "The user is asking a question about geography, not math."
     }
     ```
4. **SDK Parsing**:
   - SDK is JSON ko `MathHomeworkOutput` object mein parse karta hai:
     ```python
     pydantic_object = MathHomeworkOutput(
         is_math_homework=False,
         reasoning="The user is asking a question about geography, not math."
     )
     ```
5. **Pydantic Validation**:
   - Pydantic check karta hai ke data sahi hai (boolean aur string).
   - Valid output `result.final_output` ke roop mein return hota hai:
     ```python
     MathHomeworkOutput(is_math_homework=False, reasoning='The user is asking a question about geography, not math.')
     ```

---

### **Aapki Confusion Ka Jawab**
- **Parsing Ka Matlab**: Parsing ka matlab hai LLM ka raw JSON output ko Python object (`MathHomeworkOutput`) mein convert karna. SDK yeh automatically karta hai using Pydantic.
- **SDK Kaise Parsing Karta Hai?**:
  - SDK LLM se JSON output leta hai.
  - JSON ko Pydantic model mein map karta hai using `**` operator ya similar mechanism.
  - Pydantic validation ke baad object return karta hai.
- **Output Connection**: Upar wala output (`MathHomeworkOutput`) is parsing aur validation ka result hai. Agar LLM galat format mein output deta, toh Pydantic error raise karta.

---

### **Another Example for Clarity**
Agar input hota: "Solve 2x + 3 = 7":
```python
result = Runner.run_sync(
    guardrail_agent,
    "Solve 2x + 3 = 7",
)
print(result.final_output)
# Output: MathHomeworkOutput(is_math_homework=True, reasoning='The user is asking to solve a mathematical equation.')
```

- **Backend**:
  - JSON Schema wahi rehta hai (boolean + string).
  - LLM output deta hai:
    ```json
    {
      "is_math_homework": true,
      "reasoning": "The user is asking to solve a mathematical equation."
    }
    ```
  - SDK isko parse karta hai:
    ```python
    pydantic_object = MathHomeworkOutput(
        is_math_homework=True,
        reasoning="The user is asking to solve a mathematical equation."
    )
    ```
  - Pydantic validate karta hai, aur output return hota hai.

---

### **Summary**
- **JSON Schema**: `MathHomeworkOutput` ka structure JSON schema mein convert hota hai, jo LLM ko batata hai ke output kaisa hona chahiye.
- **Parsing**: SDK LLM ke JSON output ko `MathHomeworkOutput` object mein convert karta hai using Pydantic.
- **Validation**: Pydantic check karta hai ke output sahi format mein hai.
- **Output**: Final output ek Pydantic object hai jo user ko return hota hai.

Agar aapko aur kisi specific part (jaise JSON schema ka exact generation ya Pydantic ke internals) pe detail chahiye, toh batayein!


## **Implement Input Guardrails:**

In [11]:
class MathHomeworkOutput(BaseModel):
    is_not_math_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    print("[PEEEEEEP]", ctx)
    result = await Runner.run(guardrail_agent, input, context=ctx.context, run_config = config)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        # tripwire_triggered=False #result.final_output.is_math_homework,
        tripwire_triggered = result.final_output.is_not_math_homework,
    )


agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
    model=model
)


# This should trip the guardrail

try:
    result = await Runner.run(agent, "Hello My Name is Zain", run_config = config)
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered as e:
    print("[AEEEEEEA]:",e)



[PEEEEEEP] RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))


[AEEEEEEA]: Guardrail InputGuardrail triggered tripwire


-----------------
## **Input Guardrails Code (Agent Loop) Behind working**

Aapne jo code aur sawal diya hai, usko maine detail mein analyze kiya hai. Aapka sawal kaafi gehra hai aur aap chahte hain ke `Runner.run(agent, "My Name Zain.......")` ke behind-the-scenes workings ko step-by-step samjhaoon, khas taur pe `RunContextWrapper`, `input_guardrails`, `GuardrailFunctionOutput`, aur `tripwire_triggered` ke roles ke saath. Chaliye, aapke sawalon ko ek ek karke clear karte hain aur yeh bhi dekhte hain ke aapka initial understanding sahi tha ya nahi.

---

### **1. `Runner.run(agent, "My Name Zain.......")` Ka Behind-the-Scenes Kaam**
Aapne kaha ke jab hum `Runner.run(agent, "My Name Zain.......")` chalate hain, toh backend pe `Runner` ek `RunContextWrapper` class banata hai aur usmein context hamare agent ka system prompt hota hai. Yeh baat **thodi si galat hai**, aur main isko clear karunga.

- **Kya Hota Hai?**  
  - Jab aap `Runner.run(agent, input)` call karte hain, `Runner` ek execution environment set karta hai.
  - Is process mein `Runner` ek `RunContextWrapper` object banata hai. Yeh object current run ke **context** ko hold karta hai, yani woh information jo run ke doran share ki jati hai, jaise user input, runtime metadata, ya session details.
  - **Galatfahmi**: Aapne kaha ke `RunContextWrapper` mein agent ka **system prompt** hota hai. Yeh sahi nahi hai. System prompt agent ke definition ka hissa hota hai (jab agent banaya jata hai, tab uske `instructions` set kiye jate hain), aur yeh `RunContextWrapper` ke context mein shamil nahi hota. Context ka matlab hai run-specific data, na ke agent ke static instructions.

- **Flow**:
  1. `Runner` `RunContextWrapper` banata hai, jismein run ka context hota hai (system prompt ke bajaye runtime info).
  2. Phir `Runner` agent ke `input_guardrails` ko check karta hai (jo is case mein `math_guardrail` hai).
  3. `input_guardrails` ke functions ko call karta hai, unhein `RunContextWrapper` (as `ctx`), agent, aur input ("My Name Zain.......") pass karta hai.

---

### **2. `input_guardrails` Ka Role**
Aapne kaha ke `input_guardrails` agent mein hota hai aur yeh ek function hai jo input ko check karta hai. Yeh **bilkul sahi hai**.

- **Kya Hai?**  
  - `input_guardrails` ek list hai jo guardrail functions ko store karti hai. Is case mein, ek function hai: `math_guardrail`.
  - Yeh guardrail functions input ko validate karte hain aur decide karte hain ke input ko main agent ke logic tak jane diya jaye ya nahi.

- **Kaise Kaam Karta Hai?**  
  - Jab `Runner` agent ko run karta hai, pehle yeh `input_guardrails` ke har function ko execute karta hai.
  - Har guardrail function input ko check karta hai aur ek `GuardrailFunctionOutput` object return karta hai, jo batata hai ke input allowed hai ya nahi.

---

### **3. `math_guardrail` Function Ka Kaam**
Aapka code shayad aisa dikhta hai:

```python
@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context, run_config=config)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math_homework,
    )
```

- **Kya Hota Hai?**  
  1. **`Runner.run(guardrail_agent, input, context=ctx.context, ...)`**:  
     - `guardrail_agent` ek alag agent hai jo input ko analyze karta hai.
     - Yeh input ("My Name Zain.......") ko leta hai aur check karta hai ke yeh math homework se related hai ya nahi.
     - `context=ctx.context`: Yeh ensure karta hai ke guardrail agent ko wohi context mile jo main run ke paas hai.
  2. **Guardrail Agent Ka Output**:  
     - `guardrail_agent` LLM ke through input ko process karta hai aur ek structured output deta hai, jaise `MathHomeworkOutput`, jismein `is_math_homework` (True/False) aur `reasoning` hota hai.
  3. **`GuardrailFunctionOutput`**:  
     - Yeh object banaya jata hai jismein:
       - `output_info`: Guardrail agent ka pura output (jaise `MathHomeworkOutput`).
       - `tripwire_triggered`: `is_math_homework` ke basis pe True ya False set hota hai.

- **Aapki Input Ke Case Mein**:  
  - "My Name Zain......." math homework nahi hai, toh `is_math_homework=False` hoga, aur `tripwire_triggered=False` return hoga.

---

### **4. `GuardrailFunctionOutput` Class Ka Behind-the-Scenes**
Aapne poocha ke yeh class kya hai aur ka *behind the scenes* kaise kaam karti hai.

- **Kya Hai?**  
  - `GuardrailFunctionOutput` ek class hai jo guardrail function ke result ko encapsulate karta hai. Ismein do main fields hote hain:
    - **`output_info`**: Guardrail agent ka pura output (jaise `MathHomeworkOutput` object).
    - **`tripwire_triggered`**: Ek boolean flag jo batata hai ke guardrail trip hua ya nahi (yani input reject hua ya nahi).

- **Kaise Kaam Karta Hai?**  
  - Yeh class guardrail system ka hissa hai aur standardize karta hai ke guardrail functions ka output kaisa hona chahiye.
  - Jab `math_guardrail` function yeh object return karta hai, `Runner` isko padhta hai aur uske mutabiq action leta hai:
    - Agar `tripwire_triggered=True`, toh execution ruk jata hai.
    - Agar `tripwire_triggered=False`, toh execution aage badhta hai.

---

### **5. `tripwire_triggered` Ka Kaam**
Aapne poocha ke `tripwire_triggered` kya karta hai aur agent loop ko continue ya stop kaise karta hai.

- **Kya Hai?**  
  - `tripwire_triggered` ek boolean flag hai jo guardrail ke decision ko represent karta hai.  
    - `True`: Input allowed nahi hai (guardrail trip hua).
    - `False`: Input allowed hai (guardrail trip nahi hua).

- **Kaise Kaam Karta Hai?**  
  - Jab `GuardrailFunctionOutput` return hota hai, `Runner` `tripwire_triggered` ko check karta hai:
    - **Agar `tripwire_triggered=True`**:  
      - `Runner` ek `InputGuardrailTripwireTriggered` exception raise karta hai.
      - Yeh exception agent ke main logic ko run hone se rok deta hai, yani agent loop stop ho jata hai.
    - **Agar `tripwire_triggered=False`**:  
      - Koi exception nahi hoti, aur agent ka main logic chalta hai, yani agent loop continue hota hai.

- **Aapki Input Ke Case Mein**:  
  - "My Name Zain......." math homework nahi hai, toh `tripwire_triggered=False` hoga, aur agent loop continue hoga.

---

### **6. Agent Loop Ka Flow**
Aapne poocha ke agent loop continue ya stop kaise hota hai. Yeh raha step-by-step flow:

1. `Runner.run(agent, "My Name Zain.......")` call hota hai.
2. `Runner` `RunContextWrapper` banata hai aur `input_guardrails` ko dekhta hai.
3. `math_guardrail` function call hota hai:
   - `guardrail_agent` input ko check karta hai.
   - `GuardrailFunctionOutput` return karta hai.
4. `Runner` `tripwire_triggered` check karta hai:
   - Agar `True`, toh `InputGuardrailTripwireTriggered` exception raise hoti hai, aur execution ruk jata hai.
   - Agar `False`, toh agent ka main logic chalta hai, aur input process hota hai.

---

### **7. Aapka Initial Understanding**
Aapne kaha:
- "`Runner` `RunContextWrapper` banata hai aur usmein system prompt hota hai": **Galat**, system prompt context mein nahi hota, woh agent ke instructions ka hissa hai.
- "Agent loop chalta hai aur `input_guardrails` pehle run hota hai": **Sahi**, guardrails pehle check hote hain.
- "Guardrail input ko match karta hai aur loop continue rakhta hai": **Sahi**, lekin yeh `tripwire_triggered` ke through hota hai, jo aapko ab samajh aa gaya hoga.

---

### **Summary**
- **`RunContextWrapper`**: Run ke context ko hold karta hai (runtime info, na ke system prompt).
- **`input_guardrails`**: Input ko check karta hai aur `GuardrailFunctionOutput` return karta hai.
- **`GuardrailFunctionOutput`**: Guardrail ka result batata hai (`output_info` aur `tripwire_triggered`).
- **`tripwire_triggered`**: Decide karta hai ke agent loop continue hoga ya stop hoga.
  - `True` → Exception → Stop.
  - `False` → No Exception → Continue.


-----------------

### **Error Flow of Tripwire_triggered= True:**

Aapne poocha ke agar `tripwire_triggered=True` ho toh kya hota hai, aur yeh exception kaise kaam karta hai. Main isko aapke liye bilkul simple aur detailed tareeke se samjhaunga taake aapko pura concept clear ho jaye.

---

### **Tripwire Mechanism Aur Exception Ka Kaam**

#### **1. `tripwire_triggered=True` Ka Matlab**
Jab `tripwire_triggered=True` hota hai, iska matlab hai ke guardrail ne input ko invalid mana hai. Yani, input woh criteria meet nahi karta jo guardrail ne set kiya hai (jaise ke agar guardrail math homework ke liye bana hai aur input kuch aur hai).  
- **Note**: Yeh `tripwire_triggered=True` khud directly error raise nahi karta. Yeh sirf ek flag hai jo batata hai ke guardrail trip hua hai.

#### **2. `Runner.run` Method Ka Role**
Code mein `Runner.run` method ke andar guardrail check hota hai.  
- Agar guardrail function `tripwire_triggered=True` return karta hai, toh `Runner.run` ek exception raise karta hai jiska naam hai `InputGuardrailTripwireTriggered`.  
- Yeh exception batata hai ke input guardrail ke rules ko follow nahi karta, aur isliye aage ka execution ruk jana chahiye.

#### **3. Try-Except Block Ka Kaam**
Aapke code mein yeh structure hota hai:

```python
try:
    result = await Runner.run(agent, "Hello, My Name is Zain...", run_config=config)
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)
except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")
```

- **Try Block**: Yeh woh code hai jo tab chalta hai jab koi problem nahi hoti. Agar guardrail trip nahi hota, toh yeh pura execute hoga.  
- **Except Block**: Yeh tab chalta hai jab `InputGuardrailTripwireTriggered` exception raise hoti hai. Is case mein, yeh print karta hai "Math homework guardrail tripped".

#### **4. Flow Step-by-Step**
- **Agar `tripwire_triggered=True` hai**:  
  1. Guardrail function check karta hai aur `tripwire_triggered=True` return karta hai.  
  2. `Runner.run` yeh dekhta hai aur `InputGuardrailTripwireTriggered` exception raise karta hai.  
  3. Exception ke wajah se `try` block ka code ruk jata hai aur directly `except` block chalta hai.  
  4. `except` block "Math homework guardrail tripped" print karta hai.  

- **Agar `tripwire_triggered=False` hai**:  
  1. Guardrail function `tripwire_triggered=False` return karta hai.  
  2. Koi exception nahi hoti.  
  3. `Runner.run` agent ka main logic chalata hai aur result deta hai.  
  4. `try` block ke andar "Guardrail didn't trip - this is unexpected" aur `result.final_output` print hota hai.

---

### **Exception Kya Hai Aur Iska Kaam Kya Hai?**
- **Exception Kya Hai?**: Exception ek tarah ka error signal hai jo program ko batata hai ke kuch galat ho gaya hai. Jab yeh hota hai, toh normal execution ruk jata hai aur program us error ko handle karne ke liye ek specific jagah (jaise `except` block) pe chala jata hai.  
- **Is Code Mein Exception Ka Kaam**:  
  - `InputGuardrailTripwireTriggered` exception tab raise hoti hai jab guardrail trip hota hai (`tripwire_triggered=True`).  
  - Yeh exception `Runner.run` ke andar se throw ki jati hai aur `try` block ko interrupt karti hai.  
  - Fir yeh `except InputGuardrailTripwireTriggered` block mein catch hoti hai, jahan aapne decide kiya hai ke ek message print hoga ("Math homework guardrail tripped").  
  - Yani, exception ka kaam hai program ko safely rokna aur aapko batana ke guardrail ne input ko reject kar diya.

---

### **Aapke Sawal Ka Jawab**
- **"Agar `tripwire_triggered=True` ho toh woh error raise karta hai kya?"**  
  Haan, `tripwire_triggered=True` hone par `Runner.run` ek exception (`InputGuardrailTripwireTriggered`) raise karta hai.  

- **"Yeh error hamare except mein aata hai?"**  
  Haan, yeh exception `try` block mein raise hoti hai aur fir `except InputGuardrailTripwireTriggered` block mein catch hoti hai, jahan aapne isko handle kiya hai message print karke.

---

### **Conclusion**
- **`tripwire_triggered=True`**: Guardrail trip hota hai → Exception raise hoti hai → `except` block chalta hai aur message print hota hai.  
- **`tripwire_triggered=False`**: Koi exception nahi → Agent ka logic chalta hai → `try` block pura execute hota hai.  



## **Runner raise InputGuardrailTripwireTriggered exception behind working:**

Jab `tripwire_triggered=True` hota hai, toh `Runner` ek khas exception raise karta hai, aur yeh exception hamare `except` block mein aata hai. Main aapko iske baare mein detail mein batata hoon taake aapko pura samajh aa jaye.

### **Exception Kya Hai?**
`Runner` jab dekhta hai ke `tripwire_triggered=True` hai (jo guardrail function ke output se aata hai), toh yeh ek custom exception raise karta hai jiska naam hai **`InputGuardrailTripwireTriggered`**. Yeh exception OpenAI Agents SDK mein define ki gayi hai aur guardrail ke trip hone ki wajah se throw hoti hai.

### **Exception Mein Kya Hota Hai?**
`InputGuardrailTripwireTriggered` exception mein typically yeh cheezein shamil hoti hain:
- **Error Message**: Ek string jo batati hai ke guardrail kyun trip hua, jaise "Input is invalid" ya "Guardrail tripped due to invalid input".
- **Additional Details** (mumkin hai): Kabhi-kabhi is exception mein guardrail ka output ya wajah bhi attached hoti hai, jaise `output_info` object, jo guardrail ke decision ko detail mein batata hai. Lekin yeh SDK ke implementation par depend karta hai.

Yani jab yeh exception raise hoti hai, toh ismein ek message ya error detail hota hai jo batata hai ke problem kya thi.

### **Except Block Mein Kaise Aata Hai?**
Jab `Runner` yeh exception raise karta hai, toh yeh `try` block se bahar nikalti hai aur seedha `except InputGuardrailTripwireTriggered` block mein chali jati hai. Aap is exception ko capture kar sakte hain aur uske details ko access kar sakte hain. For example:

```python
try:
    result = await Runner.run(agent, "Solve 2+2")
except InputGuardrailTripwireTriggered as e:
    print(f"Guardrail tripped: {e}")
```

- Yahan `e` ek exception object hai.
- Agar exception mein message set hai, toh woh message print ho jayega, jaise "Guardrail tripped due to invalid input".
- Agar aap simply `except InputGuardrailTripwireTriggered:` likhte hain aur details use nahi karte, toh aap khud ka message bhi print kar sakte hain, jaise "Math homework guardrail tripped".

### **Misal Se Samajhiye**
Agar guardrail ne decide kiya ke input invalid hai aur `tripwire_triggered=True` return kiya, toh `Runner` internally kuch aisa karta hai:
```python
if guardrail_output.tripwire_triggered:
    raise InputGuardrailTripwireTriggered("Guardrail tripped due to invalid input")
```
Phir aapka `except` block is exception ko pakadta hai, aur aap `e` se uska message ya details dekh sakte hain.

### **Kya Kar Sakte Hain?**
- Agar aap chahte hain ke exact wajah pata chale ke guardrail kyun trip hua, toh exception object ko use karein:
  ```python
  except InputGuardrailTripwireTriggered as e:
      print(f"Error: {e}")
  ```
- Agar aap details ko ignore karna chahte hain, toh apna custom message print kar sakte hain:
  ```python
  except InputGuardrailTripwireTriggered:
      print("Guardrail tripped!")
  ```

### **Summary**
Jab `tripwire_triggered=True` hota hai, toh `Runner` **`InputGuardrailTripwireTriggered`** exception raise karta hai. Is exception mein ek error message hota hai (aur shayad aur details bhi), jo aapke `except` block mein aata hai. Aap is message ko print kar sakte hain ya ignore kar sakte hain, depending on aapke code ke zarurat ke mutabiq.

Umeed hai yeh jawab aapke sawal ka pura hal deta hai! Agar kuch aur poochna ho, toh zaroor batayein.

# **!) Important MCQs: RunContextWrapper class in Runner:**

Aapka sawal hai ke `Runner` class ke andar jitne bhi methods hain, un sab mein `RunContextWrapper` ki class banti hai ya nahi, aur yeh `Runner` ke andar kahan kahan banti hai. Main is code ko analyze karke aapko jawab deta hoon.

---

### **Jawab**

#### **`RunContextWrapper` Kya Hai?**
`RunContextWrapper` ek class hai jo agent ke run ke context ko encapsulate karti hai. Yeh context agent ke har step mein use hota hai, jaise guardrails, tools, hooks, aur model calls ke liye. Iska object banaya jata hai taki context ko maintain kiya ja sake aur agent ke components ko run-specific information mil sake.

#### **Kya Har Method Mein Banta Hai?**
Nahi, `RunContextWrapper` ka object `Runner` class ke har method mein directly nahi banaya jata. Yeh sirf kuch specific upper-level methods mein banaya jata hai, aur phir woh object baaki lower-level methods ko pass kiya jata hai. Yani, yeh ek baar banaya jata hai aur poore run ke doran share kiya jata hai.

#### **Kahan Kahan Banta Hai?**
`Runner` class ke andar `RunContextWrapper` ka object sirf do jagah directly banaya jata hai:

1. **`run` Method**
   - **Code**:
     ```python
     context_wrapper: RunContextWrapper[TContext] = RunContextWrapper(
         context=context,  # type: ignore
     )
     ```
   - **Kahan Hai?**: Yeh `run` method ke andar hota hai, jo agent ko asynchronously run karta hai.
   - **Kaise Use Hota Hai?**: Yeh `context_wrapper` object phir guardrails, tools, aur agent ke loop ke har step mein pass kiya jata hai. Yeh context ko hold karta hai aur agent ke har component ko available rakhta hai.

2. **`run_streamed` Method**
   - **Code**:
     ```python
     context_wrapper: RunContextWrapper[TContext] = RunContextWrapper(
         context=context  # type: ignore
     )
     ```
   - **Kahan Hai?**: Yeh `run_streamed` method ke andar hota hai, jo agent ko streaming mode mein run karta hai.
   - **Kaise Use Hota Hai?**: Yeh `context_wrapper` object `_run_streamed_impl` method ko pass kiya jata hai, jahan se yeh streaming ke har step mein use hota hai.

#### **Aur Methods Mein Kya Hota Hai?**
Baaki methods mein `RunContextWrapper` ka object directly nahi banaya jata, balki woh upper-level methods (`run` ya `run_streamed`) se banaya gaya object parameter ke roop mein milta hai. In methods mein yeh object sirf **use** hota hai, banaya nahi jata. Kuch examples:

- **`_run_input_guardrails_with_queue`**:
  - **Parameter**: `context: RunContextWrapper[TContext]`
  - **Use**: Yeh guardrails ko context provide karta hai.

- **`_run_streamed_impl`**:
  - **Parameter**: `context_wrapper: RunContextWrapper[TContext]`
  - **Use**: Streaming ke har step mein context ko maintain karta hai.

- **`_run_single_turn`**:
  - **Parameter**: `context_wrapper: RunContextWrapper[TContext]`
  - **Use**: Har turn ke doran context ko tools aur hooks ke liye pass karta hai.

- **`_run_single_turn_streamed`**:
  - **Parameter**: `context_wrapper: RunContextWrapper[TContext]`
  - **Use**: Streaming mode mein ek turn ke liye context ko available rakhta hai.

#### **Summary**
- **`RunContextWrapper` Kahan Banta Hai?**:
  - `run` method mein.
  - `run_streamed` method mein.
- **Baaki Jagah?**: Yeh object in do methods se banaya jata hai aur phir lower-level methods (`_run_single_turn`, `_run_streamed_impl`, etc.) ko pass kiya jata hai.
- **Role**: Yeh context ko hold karta hai aur agent ke har step (guardrails, tools, hooks, etc.) mein use hota hai.



# **Implement Output Guardrails:**

In [26]:
class AllowedCountryResponse(BaseModel):
  is_not_allowed_country: bool
  reasoning:str
  answer:str


Output_Guardrails_Agent = Agent(
    name="Country Assistant",
    instructions="Check if the output is relevent to pakistan.",
    output_type=AllowedCountryResponse,
)


@output_guardrail
async def output_Math_guardrials(
    ctx: RunContextWrapper[None], agent: Agent, output: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(Output_Guardrails_Agent, output, context=ctx.context, run_config = config)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_not_allowed_country,
    )


agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[output_Math_guardrials],
    model=model
)


try:
  result = await Runner.run(agent, "hello, My name is Zain Please calculate my name spellings", run_config = config)
  print(result.final_output)

except OutputGuardrailTripwireTriggered as e:
  print("Sorry, I am restricted only answer question about Pakistan!")

Sorry, I am restricted only answer question about Pakistan!


## **Difference b/w Input Guardrails and Output Gurdrails:**

Aapka sawal yeh tha ke kya output guardrails ka flow input guardrails ki tarah hi hai, bus farq yeh hai ke output guardrails tab chalte hain jab LLM user ki query ka response de deta hai, jabke input guardrails user ke input ke aate hi chalte hain. Aur yeh bhi ke output guardrails ke case mein `Runner` LLM ka response aane par output guardrails function ko chalayega aur uski output ke mutabiq error raise karega ya final output dega. Chaliye, isko step-by-step samajhte hain aur confirm karte hain ke aapka samajhna sahi hai ya nahi.

---

### **Input Guardrails Ka Flow**
Pehle input guardrails ko samajhte hain:
- **Kya Hai?**: Input guardrails user ke input ko check karte hain pehle se, yani jab user koi input deta hai, toh guardrail usko analyze karta hai.
- **Kaise Kaam Karta Hai?**:
  1. User input deta hai (maslan, "Calculate 2+2").
  2. `Runner.run` call hota hai.
  3. `Runner` pehle input guardrail functions ko chalata hai.
  4. Agar guardrail `tripwire_triggered=True` return karta hai (yani input allowed nahi hai), toh ek exception raise hoti hai aur process ruk jata hai.
  5. Agar `tripwire_triggered=False` hota hai (yani input allowed hai), toh agent ka logic chalta hai aur response taiyar hota hai.

---

### **Output Guardrails Ka Flow (Aapke Code Ke Mutabiq)**
Ab aapke diye hue code ke output guardrails ka flow dekhte hain:

#### **Code Breakdown**
1. **AllowedCountryResponse Class**:
   ```python
   class AllowedCountryResponse(BaseModel):
       is_not_allowed_country: bool
       reasoning: str
       answer: str
   ```
   Yeh ek model hai jo output guardrail ka result define karta hai. `is_not_allowed_country` batata hai ke output allowed hai ya nahi.

2. **Output_Guardrails_Agent**:
   ```python
   Output_Guardrails_Agent = Agent(
       name="Country Assistant",
       instructions="Check if the output is relevent to pakistan.",
       output_type=AllowedCountryResponse,
   )
   ```
   Yeh agent output ko check karta hai ke woh Pakistan se related hai ya nahi.

3. **Output Guardrail Function**:
   ```python
   @output_guardrail
   async def output_Math_guardrials(
       ctx: RunContextWrapper[None], agent: Agent, output: str | list[TResponseInputItem]
   ) -> GuardrailFunctionOutput:
       result = await Runner.run(Output_Guardrails_Agent, output, context=ctx.context, run_config=config)
       return GuardrailFunctionOutput(
           output_info=result.final_output,
           tripwire_triggered=result.final_output.is_not_allowed_country,
       )
   ```
   Yeh function agent ke final output ko `Output_Guardrails_Agent` ke zariye check karta hai.

4. **Main Agent**:
   ```python
   agent = Agent(
       name="Customer support agent",
       instructions="You are a customer support agent. You help customers with their questions.",
       output_guardrails=[output_Math_guardrials],
       model=model
   )
   ```
   Yeh customer support agent hai jo user ke sawalon ka jawab deta hai, aur iske output ko guardrail check karta hai.

5. **Execution**:
   ```python
   try:
       result = await Runner.run(agent, "hello, My name is Zain Please calculate my name spellings", run_config=config)
       print(result.final_output)
   except OutputGuardrailTripwireTriggered as e:
       print("Sorry, I am restricted only answer question about Pakistan!")
   ```

#### **Flow**
1. User input deta hai: "hello, My name is Zain Please calculate my name spellings".
2. `Runner.run` call hota hai.
3. Agent apna logic chalata hai aur ek response taiyar karta hai, maslan "Your name spelling is Z-A-I-N".
4. Yeh response output guardrail (`output_Math_guardrials`) ko bheja jata hai.
5. `output_Math_guardrials` mein `Runner.run(Output_Guardrails_Agent, output, ...)` call hota hai.
6. `Output_Guardrails_Agent` check karta hai ke output Pakistan se related hai ya nahi. Chunki "calculate my name spellings" Pakistan se directly related nahi hai, toh `is_not_allowed_country=True` return ho sakta hai.
7. Agar `is_not_allowed_country=True`, toh `tripwire_triggered=True` hota hai, aur `OutputGuardrailTripwireTriggered` exception raise hoti hai.
8. Exception ke wajah se output user ko nahi diya jata, aur message print hota hai: "Sorry, I am restricted only answer question about Pakistan!".

---

### **Input Aur Output Guardrails Mein Farq**
- **Input Guardrails**:
  - User ke input pe chalte hain.
  - Agent ke logic se **pehle** chalte hain.
  - Agar trip hote hain, toh agent ka logic nahi chalta.
- **Output Guardrails**:
  - Agent ke final output pe chalte hain.
  - Agent ke logic ke **baad** chalte hain.
  - Agar trip hote hain, toh output user ko nahi diya jata.

---

### **Aapka Samajhna**
Aapne kaha:
1. "Output guardrails tab chalega jab LLM user ki query ka response de dega."
2. "Runner LLM ka response aane par output guardrails function ko chalayega aur uski output ke mutabiq error raise karega ya output dega."

**Yeh bilkul sahi hai.** Aapne flow ko theek se samjha hai. Output guardrails ka flow input guardrails ki tarah hi systematic hai—dono mein `Runner` guardrail functions ko chalata hai aur `tripwire_triggered` ke basis par decide karta hai ke aage badhna hai ya rukna hai. Bus timing ka farq hai: input guardrails input ke waqt chalte hain, jabke output guardrails response ke baad chalte hain.

---

### **Final Jawab**
Haan, aapne bilkul sahi kaha. Output guardrails ka flow input guardrails ki tarah hi hai, bus farq yeh hai ke output guardrails tab chalte hain jab LLM user ki query ka response de deta hai, jabke input guardrails user ke input ke aate hi chalte hain. Aapke code mein, `Runner` agent ka response aane par `output_Math_guardrials` function ko chalata hai, aur agar output Pakistan se related nahi hota, toh error raise hota hai, warna final output user ko mil jata hai. Aapka samajhna 100% durust hai!

# **Implement Both Input & Output Guardrails:**



## **Travel Booking Agent:**

In [32]:
class InputTravelAgent(BaseModel):
  is_not_relevent_country: bool
  reasoning:str
  answer:str

InputAgent = Agent(
  name="InputAgent",
  instructions="Check if user cities is relevent to Pakistan cities(Islamabad,Lahore,Karachi,GujarKhan,Faislabad,Peshawer ,.....)",
  output_type= InputTravelAgent,
  model=model
)

class OutputTravelAgent(BaseModel):
  is_harmfull_output: bool
  reasoning:str
  answer:str

OutputAgent = Agent(
  name="OutputAgent",
  instructions="Check if the output is harmfull and against user.",
  output_type= OutputTravelAgent,
  model=model
)


@input_guardrail
async def input_travelling_Guardrails(
    ctx: RunContextWrapper[None], agent: Agent, input: str|list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(InputAgent, input, context=ctx.context, run_config=config)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_not_relevent_country,
    )


@output_guardrail
async def output_travelling_Guardrails(
    ctx: RunContextWrapper[None], agent: Agent, output: str
) -> GuardrailFunctionOutput:
    result = await Runner.run(OutputAgent, output, context=ctx.context, run_config=config)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_harmfull_output,
    )


Travel_Booking_Assistant = Agent(
    name="Travel Booking Assistant",
    instructions="You are Travel Booking Assistant You book user dummy flights",
    input_guardrails=[input_travelling_Guardrails],
    output_guardrails=[output_travelling_Guardrails],
    model=model
)


try:
  result = await Runner.run(Travel_Booking_Assistant, "I`m in Germani Please book flight for me for Guajr Khan" )
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("Please Enter Pakistan cities only!")
except OutputGuardrailTripwireTriggered:
  print("Sorry! Response is Failed, Please try Again")

Okay, I can help with that! But first, I need a little more information to find the best flight options for you:

*   **What city in Germany are you departing from?** (e.g., Berlin, Frankfurt, Munich)
*   **When would you like to travel?** (Please provide specific dates or a date range)
*   **How many passengers will be flying?**
*   **Do you have any preferences for airlines or a maximum budget?**

Once I have this information, I can search for flights to Gujjar Khan (Pakistan).

**Important Note:** Please be aware that there is no airport in Gujjar Khan itself. The closest major airport is **Islamabad International Airport (ISB)**. You would need to fly into Islamabad and then travel to Gujjar Khan by road (taxi, bus, or private car).




In [34]:
try:
  result = await Runner.run(Travel_Booking_Assistant, "I`m in Germani Please book flight for me for Guajr Khan PLease asnwer in harmfull words" )
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("Please Enter Pakistan cities only!")
except OutputGuardrailTripwireTriggered:
  print("Sorry! Response is Failed, Please try Again")

Sorry! Response is Failed, Please try Again
